# Бортников Павел Павлович 20215 RSA

In [20]:
import random
from math import gcd
import rsa.core as core
from hashlib import sha256

### Получение простого чисел

Символ Якоби

In [21]:
def j(a, b):
    if a == 1:
        return 1
    if a % 2 == 0:
        return j(a // 2, b) * (-1) ** (((b * b - 1) // 8) % 2)
    else:
        return j(b % a, a) * (-1) ** (((a - 1) * (b - 1) // 4) % 2)

Вероятностная проверка простоты числа

In [22]:
def is_prime(b):
    for i in range(100):
        a = random.randint(1, b - 1)
        if not (b % 2 == 1 and gcd(a, b) == 1 and (j(a, b) + b) % b == pow(a, (b - 1) // 2, b)):
            return False
    return True

Генерация простого числа

In [23]:
def random_prime(a, b):
    n = random.randint(a, b)
    while not is_prime(n):
        n = random.randint(a, b)
    return n

Генерация пары простых чисел

In [24]:
def gen_primes(a, b):
    u = random_prime(a, b)
    v = random_prime(a, b)
    return u, v


-------------------------------

Расширенный алгоритм Евклида — расширение алгоритма Евклида, которое вычисляет кроме наибольшего общего делителя (НОД) целых чисел a и b ещё и коэффициенты соотношения Безу

Коэффициенты Безу - представление наибольшего общего делителя целых чисел в виде их линейной комбинации с целыми коэффициентами

In [25]:
def egcd(a, b):
    if a == 0:
        return b, 0, 1
    else:
        g, y, x = egcd(b % a, a)
        return g, x - (b // a) * y, y

Обратное по модулю целого a — это такое целое число x, что произведение ax сравнимо с 1 по модулю m

In [26]:
def mod_inv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return (x + m) % m

----------------------------------

Генерация ключей

In [27]:
def gen_keys(key_size=128):
    p, q = gen_primes(1 << (key_size // 2 - 1), (1 << key_size // 2) - 1)

    n = p * q
    phi = (p - 1) * (q - 1)
    d = random_prime(max(p, q) + 1, n)
    e = mod_inv(d, phi)
    return (e, n), (d, n)

Основной функционал

In [28]:
def encrypt(number, public_key):
    e, n = public_key
    return pow(number, e, n)


def decrypt(number, private_key):
    d, n = private_key
    return pow(number, d, n)

---------------------------------

### Тестирование

In [29]:
public, private = gen_keys(128)
e, n = public
d, n = private
test1 = 1234567890

In [33]:
enc_my = encrypt(test1, public)
enc_stock = core.encrypt_int(test1, e, n)

print(enc_my)
print(enc_stock)

dec_my = decrypt(enc_my, private)
dec_stock = core.decrypt_int(enc_stock, d, n)

print(dec_my)
print(dec_stock)


35120103679007659543258495041308750003
35120103679007659543258495041308750003
1234567890
1234567890


Для подписания документа формально всё то же самое, только вставлять мы будем хэш, приведенный к инту.